# Topic Modeling

Read more about the theory and the syntax in these [two](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) [blogposts](https://medium.com/@pratikbarhate/latent-dirichlet-allocation-for-beginners-a-high-level-intuition-23f8a5cbad71).

In [1]:
import re
import math
import time
import nltk
import gensim
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pyLDAvis.gensim
#lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
#pyLDAvis.display(lda_display)


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

/Applications/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/AirMorena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/AirMorena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/AirMorena/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
# pyLDAvis Description
#pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package 
#extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

In [17]:
def visualize_wordcloud_dict_frequencies(dict_freqs, title, relative_scaling=0.5, max_words=100,
                                background_color='black'):
    plt.figure(figsize=(10, 10))
    wordcloud = WordCloud(width=900, height=500, max_words=max_words, relative_scaling=relative_scaling,
                          normalize_plurals=False, background_color=background_color).generate_from_frequencies(
        dict_freqs)
    plt.title(title)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [18]:
# 2 functions were created one that uses the stemmer method and another the lemmatizer
def tokenizer_stemmer(text, characters, stop_words):
    text_ = text.lower()# make everything lower case
    text_ = re.sub("\d", " ", text_)# it means convert any integer into white space

    for character in characters:
        text_ = text_.replace(character, " ")# replace character by white space

    # initialize tokenizer and tokenize text
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    list_words = tokenizer.tokenize(text_)# make a list of tokenized words

    # initialize and apply stemmer
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    text_stemm = [stemmer.stem(word) for word in list_words if word not in stop_words]
    return text_stemm

def tokenizer_lemmatizer(text, characters, stop_words):
    text_ = text.lower()# make everything lower case
    text_ = re.sub("\d", " ", text_)# it means convert any integer into white space

    for character in characters:
        text_ = text_.replace(character, " ")

    # initialize tokenizer and tokenize text
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    list_words = tokenizer.tokenize(text_)

    # initialize and apply stemmer
    lemmatizer = WordNetLemmatizer()
    text_lemm = [lemmatizer.lemmatize(word) for word in list_words if word not in stop_words]
    return text_lemm


def tokenizer_ner(text, characters, stop_words):# asssign named entity recognition to words
    # Get NER
    return pos_tag(text)

In [19]:
# Use the German stopwords.
stpwrds = stopwords.words("english")

# We added some additional characters to avoid.
characters = [',', '.', '(', ')', '!', ';', '-', ':', '...', '"',
                "''", "``", '~http', '?', '@', '#', '/', "'",
                '&', '--', '[', ']', '=', '||', '+', '_', '..',
                '%', '|', '~', 'http',"\n","—","•","“","„","–",
              "$",">","*"]

In [20]:
len(stpwrds) 

179

In [21]:
# add number to the stopwords list:
mylist= ['10',
 '100',
 '12',
 '15',
 '18',
 '20',
 '2018',
 '30',
 '50',
 '99']
#stpwrds.append(mylist) 

In [22]:
# add the elements of stpwrds to mylist that has the numbers
stpwrds.extend(mylist) 

In [23]:
# check is the numbers were added
#print (stpwrds)

## Load raw reviews and extract sample

In [10]:
# import data from local disk
data = pd.read_csv('/Users/AirMorena/Desktop/final_proj/csv/data_nlp.csv')

FileNotFoundError: [Errno 2] File b'/Users/AirMorena/Desktop/final_proj/csv/data_nlp.csv' does not exist: b'/Users/AirMorena/Desktop/final_proj/csv/data_nlp.csv'

In [ ]:
data.shape

In [ ]:
#data.head()

In [ ]:
#data['Category'].unique().tolist()

In [ ]:
# condition data with a positive sentiment
data_happy=data[data['Sentiment']=='Positive']
data_happy.shape

In [ ]:
# select from data angry
data_pos=data_happy[data_happy['Category']=='HEALTH_AND_FITNESS']

In [ ]:
data_pos.shape

In [ ]:
# condition data with a negative sentiment
data_angry=data[data['Sentiment']=='Negative']
data_angry.shape

In [ ]:
# category health and fitness to do the topic modelling
# select from data angry
data_neg=data_angry[data_angry['Category']=='HEALTH_AND_FITNESS']

In [ ]:
data_neg.shape

# Feature extraction

## TF-IDF Vectorization of Translated reviews

Term Frequency-Inverse Document Frequency
As a result, we have a vector representation which gives high value for a given term if that term occurs often in that particular document and very rarely anywhere else. If the term occurs in all the documents, idf computed would be 0. TF-IDF is the product of term-frequency and inverse document frequency.

Tf-idf(w) = tf(w)*idf(w)
The more important a word is in the document, it would get a higher tf-idf score and vice versa.

In [ ]:
# Initialise the TF-IDF vectorizer with a given min_document_frequency and max_document_frequency
# we added numbers to stoplist words, so we added this in the hyperparameters
# max_df = 1000 means "ignore terms that appear in more than 1000 documents".
# min_df = 100 means "ignore terms that appear in less than 100 documents

vectorizer = TfidfVectorizer(max_df=15, min_df=5,stop_words=stpwrds)

# Transform the data into normalized frequencies of words
# this steps generates word counts for the words in your docs
X_tfidf = vectorizer.fit_transform(data_neg['Translated_Review'])


# Build the column names dictionary -> ordered dataframe
tfidf_dict = vectorizer.vocabulary_
df_tf_idf_voc = pd.DataFrame({'column_name': list(tfidf_dict.keys()), 'column_index': list(tfidf_dict.values())})
df_tf_idf_voc = df_tf_idf_voc.sort_values(by='column_index')

# Build the matrix dataframe with the right columns
df_X_tfidf = pd.DataFrame(X_tfidf.toarray())
df_X_tfidf.columns = df_tf_idf_voc['column_name'].tolist()

In [ ]:
title="TF-IDF vector representation of all Translated reviews for app category Health and Fitness"
d_freq_tfidf = df_X_tfidf.sum().to_dict()
visualize_wordcloud_dict_frequencies(d_freq_tfidf, title, relative_scaling=0.5, max_words=1000,
                                background_color='black')

## Latent Dirichlet Allocation (LDA)

#Latent Dirichlet Allocation (LDA) is a algorithms used to discover the topics that are present in a corpus. 
#A few open source libraries exist, but if you are using Python then the main contender is Gensim. 
#https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

In [14]:
# number of topics per review?

# Tweak the two parameters below

#number of topics to predict
number_topics = 2
# number of words per bacth?
batch_size = 1000

# Topic hyperparameters
# alpha
doc_topic_prior = 1/number_topics
# Beta
topic_word_prior = 1/number_topics

# Create and fit the LDA model
lda = LatentDirichletAllocation(n_components=number_topics, 
                                doc_topic_prior=doc_topic_prior,
                                topic_word_prior=topic_word_prior,
                                batch_size=batch_size)
lda.fit(X_tfidf)

# Extract word distributions for the optimized topics from LDA model
topic0 = lda.components_[0]
topic1 = lda.components_[1]
#topic2 = lda.components_[2]
#topic3 = lda.components_[3]
words = vectorizer.get_feature_names()

NameError: name 'X_tfidf' is not defined

In [15]:
lda.get_params

<bound method BaseEstimator.get_params of LatentDirichletAllocation(batch_size=1000, doc_topic_prior=0.5,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=2, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=0.5,
             total_samples=1000000.0, verbose=0)>

In [13]:
dict(zip(words, topic0))

NameError: name 'words' is not defined

In [ ]:
# Create a dictionary between the words and the frequencies for the given topic
d_topic0 = dict(zip(words, topic0))
title="Wordcloud of word frequencies for topic 0"
visualize_wordcloud_dict_frequencies(d_topic0, title, relative_scaling=0.5, max_words=100,
                                background_color='black')

In [11]:
# Create a dictionary between the words and the frequencies for the given topic
d_topic1 = dict(zip(words, topic1))
title="Wordcloud of word frequencies for topic 1"
visualize_wordcloud_dict_frequencies(d_topic1, title, relative_scaling=0.5, max_words=100,
                                background_color='black')

NameError: name 'words' is not defined

In [12]:
# Create a dictionary between the words and the frequencies for the given topic
d_topic2 = dict(zip(words, topic2))
title="Wordcloud of word frequencies for topic 2"
visualize_wordcloud_dict_frequencies(d_topic2, title, relative_scaling=0.5, max_words=100,
                                background_color='black')

NameError: name 'words' is not defined

# Create a dictionary between the words and the frequencies for the given topic
d_topic3 = dict(zip(words, topic3))
title="Wordcloud of word frequencies for topic 3"
visualize_wordcloud_dict_frequencies(d_topic3, title, relative_scaling=0.5, max_words=100,
                                background_color='black')

In [ ]:
#